<h1 align ="center">Developing the random forest</h1>

In [ ]:
import numpy as np
from skimage.io import imread_collection
from skimage.transform import resize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h2 align ="center">Importing the pre-processed data</h2>


In [ ]:

# Convert data to numpy arrays
X = X = np.load('/content/drive/MyDrive/Copy of input_data.npy')
y = np.load('/content/drive/MyDrive/Copy of output_labels.npy')
y = y.ravel()


# Train the Random Forest Classifier


<h2 align ="center">Compressing the data of 3000 features to 50 because 3000 features caused unresolvable overfitting</h2>


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
more_compressed = pca.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(more_compressed, y, test_size=0.15, random_state=42)

<h2 align ="center">Did some recursive feature elimination because even 50 features were causing some overfitting</h2>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rfe = RFE(estimator=rf, n_features_to_select=40)

In [ ]:
rfe.fit(X_train, y_train)

RFE(estimator=RandomForestClassifier(), n_features_to_select=40)

In [ ]:
selected_features_indices = rfe.support_
X_train_selected = X_train[:, selected_features_indices]

<h2 align ="center">Training the random forest on the final output of RFE</h2>

In [ ]:
rf = RandomForestClassifier(n_estimators = 120,min_samples_split=4)

# Train the Random Forest model on the selected features
rf.fit(X_train_selected, y_train)

RandomForestClassifier(min_samples_split=4, n_estimators=120)

In [ ]:
X_test_selected = X_test[:, selected_features_indices]

In [ ]:
rf.score(X_train_selected, y_train)

1.0

In [ ]:
rf.score(X_test_selected, y_test)

0.9054959785522788

In [ ]:
from sklearn.metrics import recall_score
recall_score(rf.predict(X_test_selected), y_test)

0.9157706093189965

In [ ]:
from sklearn.metrics import precision_score
precision_score(rf.predict(X_test_selected), y_test)

0.8446280991735537

<h2 align ="center">Desirable results achieved.Now saving the random forest as well as the new pca compressor and the RFE indices</h2>

In [ ]:
import pickle
with open('/content/drive/MyDrive/Data/random_forest.pkl', 'wb') as file:
    pickle.dump(rf, file)
file.close()

In [ ]:
np.save('/content/drive/MyDrive/Data/selected_features.npy', selected_features_indices)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Data/random_forest_pca.pkl', 'wb') as file:
    pickle.dump(pca, file)
file.close()